In [6]:
import os
import pandas as pd

from entity.LightNode import LightNode
from utils.DataLoader import DataLoader
from utils.Settings import Setting
from utils.ProjectPath import ProjectPath
from entity import LightCluster, LightNode

path = ProjectPath()
setting = Setting()
dataloader = DataLoader()

cluster_path = path.panv2_cluster_path
cluster_file_template = "cluster_{}.csv"
suspicious_file_template = "cluster_{}_suspicious_nodes.csv"

ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
LOAD END NODES
LOAD CREATION INFO


/home/dustin/Projects/SerialScammer/main/utils/DataLoader.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens["token0"] = tokens["token0"].str.lower()
/home/dustin/Projects/SerialScammer/main/utils/DataLoader.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens["token1"] = tokens["token1"].str.lower()


LOAD RUG PULL INFO


In [7]:
def count_hops_from_source(source_address, paths: list):
    source_address = source_address.lower()
    if not source_address in paths:
        return 0
    return len(paths[paths.index(source_address):])


def get_nodes_by_labels(root_label, target_labels, cluster: LightCluster):
    root_label_nodes = []
    target_label_nodes = dict()
    for node in cluster.nodes.values():
        if root_label in node.labels and not any(label in LightNode.LightNodeLabel.SKIP_LABELS for label in node.labels):
            root_label_nodes.append(node)
        for l in node.labels:
            if l in target_labels:
                nodes = []
                if l in target_label_nodes.keys():
                    nodes = target_label_nodes[l]
                nodes.append(node)
                target_label_nodes[l] = nodes
    return root_label_nodes, target_label_nodes


def find_impacts(root_node, target_nodes):
    satisfied_nodes = []
    hops = []
    for node in target_nodes:
        if root_node.address in node.path:
            satisfied_nodes.append(node)
            hops.append(count_hops_from_source(root_node.address, node.path))
    return satisfied_nodes, hops


def impact_analysis(root_label, target_labels, cluster: LightCluster):
    root_label_nodes, target_label_nodes = get_nodes_by_labels(root_label, target_labels, cluster)
    results = []
    for root in root_label_nodes:
        record = LightNode.LightNode.to_sort_dict(root)
        impacts = []
        for tl in target_label_nodes.keys():
            tgn = target_label_nodes.get(tl)
            satisfied_nodes, hops = find_impacts(root, tgn)
            impacts.append({"type": tl, "count": len(satisfied_nodes), "avg_distance": sum(hops) / len(hops) if len(hops) > 0 else 0})
        record["impacts"] = impacts
        results.append(record)
    return results




In [8]:
group_id = 3
cluster = LightCluster.LightCluster(group_id)
cluster.load(cluster_path)

In [9]:
results = impact_analysis(LightNode.LightNodeLabel.BIG_CONNECTOR, [LightNode.LightNodeLabel.BIG_CONNECTOR, LightNode.LightNodeLabel.SCAMMER, LightNode.LightNodeLabel.BIG_WASHTRADER], cluster)
display(results)

[]

In [10]:
df = pd.read_csv(os.path.join(path.panv2_processed_path, "1_pair_pool_labels.csv"))
display(df.head())

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/Storage/Data/Blockchain/DEX/pancakeswap/processed/1_pair_pool_labels.csv'

In [4]:
rp = df[df["is_rp"] == 1]
rp

,pool,creator,is_rp,token0,token1,scam_token
116,0x45c0a88988f33123794DAb66c232c00e7C1DdCd5,0xecaace2c92afe113057c1bf66c78ccd08e20d249,1,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xef54802c87bb66bc61f069e3e9f5953e322cff58,0xef54802c87bb66bc61f069e3e9f5953e322cff58
119,0xC403605005e2C878914ee9FEAD840490a961782b,0x7fb0792ad88320711270f219510bc3be928293c2,1,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xd1c24bcabab5f01bcba2b44b6097ba506be67d6d,0xd1c24bcabab5f01bcba2b44b6097ba506be67d6d
143,0xaaBd4586Ed416E14EBe6321523883EC36E387a47,0xecaace2c92afe113057c1bf66c78ccd08e20d249,1,0x3a5a4d66680a165863a9295f620ca17e17e8f0c1,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0x3a5a4d66680a165863a9295f620ca17e17e8f0c1
146,0x5Ce7761F16616DC0c6a2D7A8D8B2BCc09Be7d289,0x56c6e04c7466969dd5b81afb22cc0fdd14f226eb,1,0x64b90eef53e4d13e64c65cdd755bbebec75e46bd,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0x64b90eef53e4d13e64c65cdd755bbebec75e46bd
168,0x6455c1Fd1Ffd68F2c01738625Cf2Bba7FF94A85E,0xa7a3af6dac57e853e64b6eefdef2e714e5bfaed8,1,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xc435db69d3d59d4259db436448f6da8488e08b0b,0xc435db69d3d59d4259db436448f6da8488e08b0b
...,...,...,...,...,...,...
1694041,0xC689ca958a7296b8dFA449eE76b4C7aBcb590381,0xd0d9b67048d35119cd8d74873c9e592e9a83c0d3,1,0x57120a8aec67860739543e170170496b70e49454,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0x57120a8aec67860739543e170170496b70e49454
1694046,0x5eFa2A4EBEcA6eC723878CEcD3183e2A2D7d7C69,0x5d2adb678f109559d4ddb6182e101a3e6f3edc27,1,0xb864c68eae780a3f58dd9334e240dde06525128c,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xb864c68eae780a3f58dd9334e240dde06525128c
1694048,0xaDD7571517662fFFB6509E2B4811D235A3951c2f,0x6a93b7c0b53c64199719b9fbfd596bcdcefba03c,1,0x0c9eb5306a2318f3c3e7a0170ad478643e1850d7,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0x0c9eb5306a2318f3c3e7a0170ad478643e1850d7
1694056,0xFce895D67a3953c500e47b4B61899D2D275d0F1b,0x02d3c20b9530a7248018bd0ddec46c4658e320d6,1,0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c,0xee404f329e9ab68458b63d858ddae03bb65e90e2,0xee404f329e9ab68458b63d858ddae03bb65e90e2


In [5]:
pcs = pd.read_csv(os.path.join(path.panv2_processed_path, "1_pair_scammers.csv"))
print("Pancake unique scammers", len(pcs["scammer"].unique()))
uns = pd.read_csv(os.path.join(path.univ2_processed_path, "1_pair_scammers.csv"))
print("Uniswap unique scammers", len(uns["scammer"].unique()))
print("Total unique scammers", len(uns["scammer"].unique())+len(uns["scammer"].unique()))

Pancake unique scammers 255969
Uniswap unique scammers 190958
Total unique scammers 381916


In [3]:
from tqdm import tqdm
from data_collection.AccountCollector import TransactionCollector 
uns = pd.read_csv(os.path.join(path.univ2_processed_path, "filtered_simple_rp_scammers.csv"))
print("Uniswap unique scammers", len(uns["scammer"].unique()))
collector = TransactionCollector()
scammers = uns["scammer"].unique()
scammer_pools = uns.groupby("scammer")["pool"].apply(list).to_dict()
# ntxs = []
# for scammer in tqdm(scammers):
#     nor_txs, int_txs = collector.get_transactions(scammer)
#     pools = scammer_pools[scammer]
#     ntxs.append({"scammer":scammer, "num_normal_txs": len(nor_txs), "num_pools": len(pools), "pools": ";".join(pools)})
# ntxs_df = pd.DataFrame(ntxs)
# ntxs_df.to_csv("filtered_simple_scammer_txs_size.csv", index=False)
ntxs_df = pd.read_csv("filtered_simple_scammer_txs_size.csv")
display(ntxs_df.head())

Uniswap unique scammers 176750


  8%|▊         | 13817/176750 [01:09<12:01, 225.77it/s]

0x863157a21bc7534fb57311b133dd96979a5aa880 No columns to parse from file


100%|██████████| 176750/176750 [13:07<00:00, 224.56it/s]


,scammer,num_normal_txs,num_pools,pools
0,0xe2772341c3ca68f332ec8d68d13b5ebae3dd26d8,219,1,0x2F2399AfBcF979cA3D92a3158562C9A397b28D06
1,0x6f9ff1f95bde025e3d5e4633304fa8c29ea8acf8,14,1,0x0e2129c1bDbb0AfaA6Be1ed53650B267C0FB5556
2,0xc1ff218cb1355e6c7b25e8144252cc5ea3c08d9f,16,1,0x7f6fe5c9FF1F9D46B29B79537D9ce6337127DFE0
3,0xae9943ffb0ea5959ddbf7bbe93d246c861d4c939,11,1,0x083B344D961D4Ab3823ACceC60253ec6c4bdB511
4,0xcb114805b901f7a9c38d5675272ef26459a7d805,5114,1,0x441d3A8eA2076ad7326E91f52704CBE00C911F73


In [4]:
zero_txs_scammers = ntxs_df[ntxs_df["num_normal_txs"] == 0]
display(zero_txs_scammers)

,scammer,num_normal_txs,num_pools,pools
3119,0x7059010320d07a47d2ad54db88bb89ea644c3e6f,0,2,0x5d5E650f31939741d0B3abC50e773973Bc847Cee;0xE...
13778,0x863157a21bc7534fb57311b133dd96979a5aa880,0,1,0x380ba186Af68899c8D24EF564536293e14139802
57615,0x982ef39bffcd16d9192ac064ed8584f4f977fdfc,0,1,0x136052e3f32af40B9c5073B5abd242f48DAb522A
65885,0xec6a5154ba367f393c565b3597034540ba00c836,0,2,0xd1Be2858A9babebCD4244b3c42Ca173b740474A3;0x8...
88224,0xd80a8e64f021275d758ee2f2c35a9a23b1f75df9,0,1,0x9B4fc06C5c014fDeA9b1c3a9075350ecD527ab0B
101602,0x5906892a61fb475f6e175c67559c1262d9547599,0,1,0xeBEd118dBAED2a48B301fA8b40d9C23eC1DB3878
147814,0x120649ac3f63b4d85a7329ff0547d30251e07605,0,1,0xD8497c6f7CD3b8451e959136C91bf776b55DA015


In [5]:
over_thoundsandscammers = ntxs_df[ntxs_df["num_normal_txs"] > 1000]
over_thoundsandscammers

,scammer,num_normal_txs,num_pools,pools
4,0xcb114805b901f7a9c38d5675272ef26459a7d805,5114,1,0x441d3A8eA2076ad7326E91f52704CBE00C911F73
8,0x1a87b5ccb1c8257815bd2b0a87c899fac59ca598,5799,1,0xd5662ABf455beB1d81Ba46d62a82cb3D03e471fC
10,0x21fb8e54a34f462c3642b5dc3e529206fdf7b324,1230,2,0x14583f97FD7097fd70Bc57c2a35A77EaC9D1b27c;0xC...
13,0x89e2aebc454556f60f2b6facd1746bcb58f0b4f3,2217,1,0x497f4633dDfAcfc481f8C8cfA39F49bcf45b17F8
16,0x5d63095cd6a33b626ba08d121c889bd987dbba1c,1937,4,0x3fF6d7ADD4508d258e192AADC9eD1371869a4aB9;0x9...
...,...,...,...,...
172675,0xcc9c0cc7ba8fe212126a96b7bbef8bc62c53a13f,3807,1,0xC81950F7555d596913d0aB999681b59B43BABc8b
173405,0x9f49eb2d0d0c3065357b2deb39c18aca1b4cc2b9,1777,4,0xb34CCBa001997C31f2d9f51e7fc408D0bEa779B3;0x2...
174683,0x566842a1eb3dfa231574146f86aa2718850da5e2,2501,1,0x870352716AfbDec3cE29450D57f06c3060947434
174848,0x33e6d5c35117e00d66a656fae0be45621a964df9,1557,2,0xa86af204A3892FB71D5f87AD7bE0fe77D8236406;0x4...


In [6]:
uns.groupby("pool")["scammer"].count()

pool
0x0000871C95bb027c90089f4926fD1BA82Cdd9a8B    1
0x0000B693708A0967A3014365A51282c4d189CD24    1
0x0002Bc7e2DB8Df0F26b6DB17E1F5e661Cb07ebcb    1
0x0002bC7093AeFD17032B5F9E9C2237E83DdBa267    1
0x00031554Ee74E4a6d497677e5152d94c3dBE5f46    1
                                             ..
0xffe4f11b0ed815c1a157D239224555086D2Df450    1
0xffe51f3256C3800Bc11f64839a795f22A75A6DF3    1
0xfff0E1744d45Ad8AFE810Dc0166B64fc7407718E    1
0xfff2BCbE560457041E32D75A7d274F9E56f0ab14    1
0xfff7782aDd71B4fd181e455AC9bd26b5145f439f    1
Name: scammer, Length: 193104, dtype: int64